In [20]:
import pandas as pd
#Import SpaCy
import spacy
import regex
from spacy import displacy
from collections import Counter
import copy
import stanza#
import random
from fuzzywuzzy import process
#target_nlp = spacy.load('de_core_news_sm')
IN = "input/dev.roen.df.short.tsv"
OUT = "output/masked-levenshtein-1/dev.roen.masked.tsv"
JSON = "output/masked-levenshtein-1/dev.roen.mapping.json"
entities = ["PER", "PERSON", "ORG", "ORGANIZATION", "PRODUCT", "WORK_OF_ART", "LOC"]


In [9]:
def same_enity_mask(input_csv, output_csv, output_json, entities, lang):
#Set up dict & dataframe
    d = {}
    df = input_csv
    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
    return df

In [2]:
def mask(df, source_lang, target_lang):
    print("start function")
    
    #Set up models
    stanza.download(source_lang, processors='tokenize, ner',verbose=False)
    source_nlp= stanza.Pipeline(source_lang, processors='tokenize, ner',verbose=False)
    print("end english download")
    
    stanza.download(target_lang, processors='tokenize, ner',verbose=False)
    target_nlp= stanza.Pipeline(target_lang, processors='tokenize, ner',verbose=False)
    print("end download")
    
    #read in dataframe
    #df = pd.read_csv(input_csv, sep='\t', index_col="index")
    #replace entities
    for index, row in df.iterrows():
        print("row sart")
        print(index)
        source, target = source_nlp(row.original), target_nlp(row.translation)

        source.mask_ents()
        target.mask_ents()

        df.at[index, "original"]=source.text
        df.at[index, "translation"]=target.text
    #export to csv
    print("end")
    return df 

In [46]:
def mask2(input_csv, output_csv, output_json, entities, lang):
    
#Set up dict & dataframe
    d = {}
    df = input_csv

    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                entity_count+=1
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                    entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
#Export dict to json`:
    return df 

In [18]:
def different_models_maskdf(df,output_csv, output_json, entities ):
    source_nlp = spacy.load("en_core_web_sm")
    target_nlp = spacy.load("ro_core_news_sm")
    d = {}
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source, target = source_nlp(line1), target_nlp(line2)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(random.randint(0,10000))
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
#mask target
        for Y in target.ents:
            if Y.label_ in entities:
                if Y.text not in d:
                    d[Y.text] = "NE{}".format(random.randint(0,10000))
                start, end = Y.start_char-target_adjust, Y.end_char-target_adjust
                line2 = "".join([line2[:start],d[Y.text],line2[end:]])
                target_adjust=len(Y.text)-len(d[Y.text])


#Replace row with masked version.
        df.at[index,"original"]=line1
        df.at[index, "translation"]=line2

#Export dict to json
    return df



# EN-DE
This is a notebook detaiolign the masking of the en de dataset at different levels

## dev data

In [3]:
en_de_dev_df =  pd.read_csv("../data/ro-en/dev.roen.df.short.tsv",sep="\t")


result_dev = pd.read_csv("../predicted_dfs/dev.roen.predicted.zmean.tsv", sep="\t")

en_de_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,0,"Pe 17 iunie, Pétain a declart „Cu inima grea, ...","On 17 June, Pétain declared it wholeheartedly,...","[70, 60, 51, 81, 91, 100]",75.500000,"[0.018484244414180506, -0.28229248973074994, -...",0.220466,-0.329478
1,1,"Dischetele sunt numite dischete de 4 MB, refer...",The floor coverings are called 4 MB disks rela...,"[20, 5, 35, 50, 49, 50]",34.833333,"[-1.6837764568809643, -2.152063229252937, -1.5...",-1.292432,-0.488675
2,2,Rafinatul său estetism e nu numai semnul unei ...,Its aesthetic affinity is the sign not only of...,"[70, 40, 50, 56, 77, 70]",60.500000,"[0.018484244414180506, -0.9622091222842726, -0...",-0.370566,-0.501697
3,3,A realizat mai multe antologii de poezie conte...,Has developed several contemporary poesthesia ...,"[75, 45, 70, 48, 49, 58]",57.500000,"[0.188710314543695, -0.792229964145892, -0.017...",-0.474398,-0.363895
4,4,Statele trebuiau să coopereze în desfășurarea ...,We had to cooperate with states in conducting ...,"[65, 40, 45, 63, 48, 56]",52.833333,"[-0.15174182571533398, -0.9622091222842726, -1...",-0.681017,-0.312040
...,...,...,...,...,...,...,...,...
995,995,Generalul francez Philippe Pétain considera Ar...,"This is because, on the basis of the criteria ...","[11, 1, 1, 8, 1, 1]",3.833333,"[-1.9901833831140903, -2.288046555763642, -2.9...",-2.316998,-1.113937
996,996,A fost declarat iresponsabil și a fost convoca...,He was declared irresponsible and a board of d...,"[70, 100, 90, 93, 100, 91]",90.666667,"[0.018484244414180506, 1.0775407753762953, 0.8...",0.794734,-0.319020
997,997,Dublin are o rețea de transport public ce cupr...,Dublin has a public transport network comprisi...,"[100, 98, 90, 91, 100, 89]",94.666667,"[1.0398406651912675, 1.0095491121209432, 0.838...",0.960406,-0.295920
998,998,Extrema dreaptă contestă valoarea acestei mărt...,The extreme right disputes the value of this c...,"[50, 50, 80, 100, 91, 96]",77.833333,"[-0.6624200361038775, -0.6222508060075113, 0.4...",0.299347,-0.296102


In [4]:
result_dev["error"] = abs(en_de_dev_df["z_mean"]-result_dev["predictions"])
result_dev

,text_a,text_b,labels,predictions,error
0,"Pe 17 iunie, Pétain a declart „Cu inima grea, ...","On 17 June, Pétain declared it wholeheartedly,...",0.220466,-0.619299,0.839765
1,"Dischetele sunt numite dischete de 4 MB, refer...",The floor coverings are called 4 MB disks rela...,-1.292432,-0.596069,0.696364
2,Rafinatul său estetism e nu numai semnul unei ...,Its aesthetic affinity is the sign not only of...,-0.370566,-0.776996,0.406430
3,A realizat mai multe antologii de poezie conte...,Has developed several contemporary poesthesia ...,-0.474398,-0.930403,0.456005
4,Statele trebuiau să coopereze în desfășurarea ...,We had to cooperate with states in conducting ...,-0.681017,0.420659,1.101676
...,...,...,...,...,...
995,Generalul francez Philippe Pétain considera Ar...,"This is because, on the basis of the criteria ...",-2.316998,-2.249362,0.067636
996,A fost declarat iresponsabil și a fost convoca...,He was declared irresponsible and a board of d...,0.794734,0.282447,0.512286
997,Dublin are o rețea de transport public ce cupr...,Dublin has a public transport network comprisi...,0.960406,0.610148,0.350258
998,Extrema dreaptă contestă valoarea acestei mărt...,The extreme right disputes the value of this c...,0.299347,0.363810,0.064463


In [5]:
result_dev = result_dev.sort_values(by=["error"],ascending=False)
result_dev = result_dev[result_dev.error > 0.3121]
result_dev.reset_index(inplace=True,drop=True)
result_dev

,text_a,text_b,labels,predictions,error
0,Dobridorul nu este o excepție în ceea ce prive...,The acquirer is not a deposit exception,-1.961158,-0.262487,1.698671
1,"Vârsta de absolvire a școlii este 16 ani, după...","The school age shall be 16 years, after which ...",-0.930434,0.394186,1.324620
2,"simțirii, tot cel ce primește litera în chip i...","feeling, receiving the letter in face of iudai...",0.328445,-0.973316,1.301761
3,"Serbia este membră a ONU, Consiliului Europei,...","Serbia is a member of the UN, the Council of E...",1.026494,-0.255894,1.282388
4,"În secolul V, Imperiul Roman de Apus a încetat...","In the V century, the Roman de Apus Empire cea...",-0.720420,0.556006,1.276425
...,...,...,...,...,...
495,Râul Drina formează granița naturală dintre Bo...,The River Drina forms the natural border betwe...,-0.492355,-0.177616,0.314739
496,În momentul declanșării invaziei germane pe 10...,"When the German invasion started on 10 May, th...",1.003072,0.688584,0.314488
497,Elveția are cel mai eficient sistem din lume p...,Switzerland has the most efficient system in t...,1.061147,0.747763,0.313384
498,Speranta de viata a populatiei pe perioada imp...,The population's hopeful vitis during the Roma...,-0.991296,-0.678236,0.313060


In [6]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_dev_df["original"].values.tolist())[0]
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [21]:
maskd_df = different_models_maskdf(maskd_df,OUT,JSON,entities)

In [22]:
total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_dev_df.to_csv("../stanza_results/ro-en-dev-above-mae-stanza.tsv", index = None, header=True,sep = '\t')

total_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,242,Dobridorul nu este o excepție în ceea ce prive...,The acquirer is not a deposit exception,"[15, 11, 11, 22, 24, 20]",17.166667,"[-1.8540025270104787, -1.9480882394868806, -2....",-1.961158,-0.404628
1,38,"NE5253 a școlii este 16 ani, NE4072 pot alege ...","NE7185 16 years, NE3920 pupils may choose to s...","[50, 50, 70, 20, 29, 38]",42.833333,"[-0.6624200361038775, -0.6222508060075113, -0....",-0.930434,-0.299250
2,959,"simțirii, tot cel ce primește litera în chip i...","feeling, receiving the letter in face of iudai...","[50, 50, 75, 100, 100, 100]",79.166667,"[-0.6624200361038775, -0.6222508060075113, 0.1...",0.328445,-0.503945
3,671,"Serbia este membră a NE2748, NE592NE4379SCE, P...","Serbia is a member of the UN, NE7938, NE1973, ...","[100, 98, 91, 96, 98, 100]",97.166667,"[1.0398406651912675, 1.0095491121209432, 0.881...",1.026494,-0.218736
4,898,"În secolul V, NE658 a încetat se existe, cea c...","NE2300, the Roman de Apus Empire ceased to exi...","[50, 38, 50, 53, 52, 53]",49.333333,"[-0.6624200361038775, -1.030200785539625, -0.8...",-0.720420,-0.206189
...,...,...,...,...,...,...,...,...
1492,992,Mulți cnezi și voievoizi au urcat la rangul no...,Many believers and voivoids hurt the Chernobyl...,"[20, 30, 35, 26, 12, 30]",25.500000,"[-1.6837764568809643, -1.302167438561034, -1.5...",-1.567432,-0.652554
1493,993,"Capitala și cel mai mare oraș este Bratislava,...",The capital and the largest city are Bratislav...,"[70, 70, 80, 71, 80, 100]",78.500000,"[0.018484244414180506, 0.05766582654601139, 0....",0.389708,-0.179009
1494,994,În Casa Rustică sunt amenajate două încăperi c...,Two rooms with objects of popular material cul...,"[61, 91, 75, 56, 81, 78]",73.666667,"[-0.28792268181894554, 0.7715782907272102, 0.1...",0.203727,-0.270639
1495,995,Generalul francez Philippe Pétain considera Ar...,"This is because, on the basis of the criteria ...","[11, 1, 1, 8, 1, 1]",3.833333,"[-1.9901833831140903, -2.288046555763642, -2.9...",-2.316998,-1.113937


In [23]:
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,242,Dobridorul nu este o excepție în ceea ce prive...,The acquirer is not a deposit exception,"[15, 11, 11, 22, 24, 20]",17.166667,"[-1.8540025270104787, -1.9480882394868806, -2....",-1.961158,-0.404628
1,38,"NE5253 a școlii este 16 ani, NE4072 pot alege ...","NE7185 16 years, NE3920 pupils may choose to s...","[50, 50, 70, 20, 29, 38]",42.833333,"[-0.6624200361038775, -0.6222508060075113, -0....",-0.930434,-0.299250
2,959,"simțirii, tot cel ce primește litera în chip i...","feeling, receiving the letter in face of iudai...","[50, 50, 75, 100, 100, 100]",79.166667,"[-0.6624200361038775, -0.6222508060075113, 0.1...",0.328445,-0.503945
3,671,"Serbia este membră a NE2748, NE592NE4379SCE, P...","Serbia is a member of the UN, NE7938, NE1973, ...","[100, 98, 91, 96, 98, 100]",97.166667,"[1.0398406651912675, 1.0095491121209432, 0.881...",1.026494,-0.218736
4,898,"În secolul V, NE658 a încetat se existe, cea c...","NE2300, the Roman de Apus Empire ceased to exi...","[50, 38, 50, 53, 52, 53]",49.333333,"[-0.6624200361038775, -1.030200785539625, -0.8...",-0.720420,-0.206189
...,...,...,...,...,...,...,...,...
495,89,NE9836 formează granița naturală dintre Bosnia...,The NE349 forms the natural NE8609 between Bos...,"[50, 50, 51, 65, 62, 64]",57.000000,"[-0.6624200361038775, -0.6222508060075113, -0....",-0.492355,-0.220157
496,982,În momentul declanșării invaziei germane pe 10...,"NE6908 NE4463 started on 10 May, the biggest p...","[100, 100, 91, 95, 100, 91]",96.166667,"[1.0398406651912675, 1.0775407753762953, 0.881...",1.003072,-0.211574
497,761,Elveția are cel mai eficient sistem din lume p...,Switzerland has the most efficient system in t...,"[100, 100, 100, 100, 100, 93]",98.833333,"[1.0398406651912675, 1.0775407753762953, 1.267...",1.061147,-0.175295
498,398,NE6658 viata a populatiei pe perioada imperiul...,The population's hopeful vitis during the Roma...,"[35, 50, 30, 43, 44, 50]",42.000000,"[-1.173098246492421, -0.6222508060075113, -1.7...",-0.991296,-0.381120


# Training Data

In [24]:
en_de_train_df =  pd.read_csv("../data/ro-en/train.roen.df.short.tsv",sep="\t")

In [25]:
result_train = pd.read_csv("../predicted_dfs/train.roen.predicted.zmean.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])

In [26]:
result_train

,index,text_a,text_b,predictions,labels,error
0,0,"După doar 200 de metri, ajungem la Foișorul Bă...","Only 200 metres later, we arrive at the Former...",-1.457294,-1.730965,0.273671
1,1,Guvernul numește un prefect în fiecare județ p...,The Government appoints a prefect to each coun...,0.653505,0.851625,0.198120
2,2,"În 1979, sovieticii au invadat Afghanistan, ia...","In 1979, the Soviets invaded Afghanistan and t...",0.625578,1.060160,0.434582
3,3,O treime din casele sale au fost avariate sau ...,One third of his homes have been damaged or de...,0.665354,0.744143,0.078789
4,4,Înaintea introducerii tramvaiului la Galați se...,Before the tram was introduced in Galats it wo...,0.034284,0.634248,0.599964
...,...,...,...,...,...,...
6995,6995,"Pentru restul campaniei, belgienii au executat...","For the rest of the campaign, the Belgians hav...",0.285150,0.389310,0.104160
6996,6996,Una din cele mai veche publicație românească d...,One of the oldest Romanian publication in Târg...,0.518699,0.985450,0.466751
6997,6997,"ionic, elaborat în aceeași perioadă, este mai ...","The ion, developed during the same period, is ...",-0.396794,-0.291348,0.105447
6998,6998,Soldați aliați au sosit în Arhanghelsk și în V...,Aliases arrived in Arhanghelsk and Vladivostok.,-0.842558,-0.291348,0.551210


In [27]:
result_train = result_train.sort_values(by=["error"],ascending=False)
result_train = result_train[result_train.error > 0.3121]
result_train
result_train.reset_index(inplace=True,drop=True)
result_train

,index,text_a,text_b,predictions,labels,error
0,3836,"Ieșind victorios din conflicte, Regatul Unit a...",The UK managed to influence world events decis...,0.204638,-1.532969,1.737606
1,3584,Han și Chewbacca îl găsesc pe Lando și îl prov...,Han and Chewbacca find on the Lando and bring ...,-0.744584,0.902383,1.646967
2,5216,Iacobiții au fost în cele din urmă înfrânți în...,The Greeks were finally defeated in 1746 after...,-0.466400,1.054952,1.521352
3,4883,Aceste forțe urmau să înainteze în Olanda spre...,These forces were going to go to Breda in the ...,0.361675,-1.122214,1.483890
4,4491,Biggin Hill nu a fost scos din funcțiune nicio...,Biggin Hill was never brought into operation d...,0.318655,-1.084845,1.403500
...,...,...,...,...,...,...
2034,1217,", fondat în 1967, este unul dintre cele mai ve...",", founded in 1967, he is one of the world's ol...",0.041720,-0.270926,0.312646
2035,6778,"Poziția geografică favorabilă, împreună cu dez...","The favourable geographical position, together...",0.744498,1.056841,0.312343
2036,5531,thumbCorpul magazinului Manea Ionuț care nu ac...,Thumbhus Manea Ionuret shop not operating at t...,-0.747119,-0.434781,0.312337
2037,4578,Alte grupări erau mai tradiționaliste și criti...,Other groups were more traditional and critici...,0.721154,1.033464,0.312310


In [28]:

maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_train_df["original"].values.tolist())[0]
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [29]:
maskd_df = different_models_maskdf(maskd_df,OUT,JSON,entities)

In [30]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../stanza_results/ro-en-train-above-mae-stanza.tsv", index = None, header=True,sep = '\t')

total_train_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,3836,"Ieșind victorios din conflicte, NE6125 a reuși...",NE7938 managed to influence world events NE811...,"[25, 21, 16]",20.666667,"[-1.3558846703605294, -1.6773772406382186, -1....",-1.532969,-0.289111
1,3584,Han și Chewbacca îl găsesc pe Lando și îl NE41...,Han and Chewbacca find on the Lando and bring ...,"[100, 92, 96]",96.000000,"[0.8333627005754242, 0.7728312406438095, 1.100...",0.902383,-0.442212
2,5216,Iacobiții au fost în cele din urmă înfrânți în...,NE2613 were finally defeated in 1746 NE5828 we...,"[97, 100, 93]",96.666667,"[1.3343471841120664, 0.8333627005754242, 0.997...",1.054952,-0.367649
3,4883,Aceste forțe urmau să înainteze în NE3574 spre...,These forces were going to go to Breda in the ...,"[19, 45, 32]",32.000000,"[-1.3619336320056, -0.6769281428141485, -1.327...",-1.122214,-0.315221
4,4491,NE7869 nu a fost scos din funcțiune niciodată ...,"NE507 operation during the Battle of England, ...","[36, 45, 17]",32.666667,"[-0.9752473803507731, -0.7413958401276388, -1....",-1.084845,-0.215645
...,...,...,...,...,...,...,...,...
9032,6993,thumbrightVehicul blindat [[Mowag Piranha apar...,"As a result, the number of employees employed ...","[11, 10, 1]",7.333333,"[-1.9901833831140903, -1.9820840711145566, -2....",-2.315180,-0.869105
9033,6994,"Italienii finalizaseră procesul de unificare, ...",The Italians had completed the unification pro...,"[50, 50, 80]",60.000000,"[-0.6624200361038775, -0.6222508060075113, 0.4...",-0.291348,-0.312157
9034,6995,"Pentru restul campaniei, belgienii au executat...","For the rest of the campaign, the Belgians hav...","[95, 65, 80]",80.000000,"[0.8696145950617529, -0.11231333159236928, 0.4...",0.389310,-0.333843
9036,6997,"ionic, elaborat în aceeași perioadă, este mai ...","The ion, developed during the same period, is ...","[50, 50, 80]",60.000000,"[-0.6624200361038775, -0.6222508060075113, 0.4...",-0.291348,-0.376757


# same enity recogniser

In [5]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")


result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])

result_dev = result_dev.sort_values(by=["error"],ascending=False)

result_dev= result_dev[result_dev["error"]>= 0.6444].reset_index(drop=True)
result_dev

,text_a,text_b,labels,predictions,error
0,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,-4.825809,-0.340117,4.485691
1,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,-3.760660,-0.349047,3.411613
2,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...",-3.730812,-0.373251,3.357561
3,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,-3.760030,-0.449195,3.310835
4,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,-3.616978,-0.311670,3.305308
...,...,...,...,...,...
522,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,0.265726,-0.383941,0.649667
523,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...",0.335268,-0.313811,0.649078
524,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,0.376660,-0.270235,0.646895
525,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...",0.320917,-0.325518,0.646435


In [6]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        pass
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
518,211,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,"[91, 95, 92, 84, 95, 71]",88.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.265726,-0.550166
519,574,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...","[91, 95, 92, 100, 76, 92]",91.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.335268,-0.381236
520,376,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,"[100, 95, 95, 100, 96, 73]",93.166667,"[0.9278538515065428, 0.7925229148025456, 0.746...",0.376660,-0.276205
521,278,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...","[90, 70, 95, 92, 78, 100]",87.500000,"[0.506574444399416, -0.19864283100263583, 0.74...",0.320917,-0.371713


In [10]:
maskd_df = same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

In [11]:
total_dev_df.to_csv("../masked-data/masked_dev_en_de_same.tsv", index = None, header=True,sep = '\t')

total_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,NE0.,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In NE1.,In NE1.,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, NE2 w...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The NE3 tied it in the third on a triple by NE...,Die NE3 band es in der dritten auf einem Tripl...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,MNE6erweise wohnten alte südarabische Sprecher...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
1514,991,"She earned high grades, excelling in French an...","Sie verdiente hohe Noten, die sich im Französi...","[91, 80, 88, 87, 88, 88]",87.000000,"[0.5487023851101287, 0.19782346731943673, 0.45...",0.359279,-0.636871
1516,993,"Trio d'anches, or trio of reeds consists of ob...",Trio d 'anches oder Trio aus Schilf besteht au...,"[50, 50, 50, 100, 82, 72]",67.333333,"[-1.1785431840290908, -0.991575427646781, -1.1...",-0.630671,-0.198255
1518,995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,"[70, 65, 95, 100, 77, 100]",84.500000,"[-0.33598436981483737, -0.3968759801636721, 0....",0.088897,-0.259579
1520,997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,"[50, 50, 40, 100, 84, 92]",69.333333,"[-1.1785431840290908, -0.991575427646781, -1.5...",-0.484296,-0.412556


In [15]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])


In [16]:
result_train = result_train.sort_values(by=["error"],ascending=False)
result_train= result_train[result_train["error"]>= 0.6444].reset_index(drop=True)

In [17]:
maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        pass
    
maskd_df =  same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

In [18]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_same.tsv", index = None, header=True,sep = '\t')

total_train_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,6975,He vacated the NE0 title in April 2002 after r...,Nach seiner Rückkehr zum NE1 verließ er im Apr...,"[1, 13, 24]",12.666666666666666,"[-9.000885564931252, -10.29283318973243, -3.33...",-7.541931,-0.360573
1,2080,"NE2 and NE3, Proceedings of International Coll...","NE2 and NE3, Proceedings of International Coll...","[1, 1, 1]",1.0,"[-9.000885564931252, -8.00864789335575, -4.573...",-7.194496,-0.240925
2,6229,NE4 North Star Osage Ozark Trailblazers Pathfi...,NE4 North Star Osage Ozark Trailblazers Pathfi...,"[1, 1, 1]",1.0,"[-9.000885564931252, -7.556217303171075, -4.57...",-7.043685,-0.061312
3,2214,NE5.,NE5.,"[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.220932
4,1846,"NE6, NE7, IFWG PublishinNE8","NE6, NE7, NE8.","[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.129047
...,...,...,...,...,...,...,...,...
9964,6992,Those attacks exacted a toll from Beales neigh...,Diese Angriffe forderten eine Maut von Beales ...,"[100, 91, 75, 50, 39, 35]",65.0,"[0.2654514735602496, 0.04515183701325695, -0.5...",-0.611383,-0.342817
9965,6993,"Thus, even a smaller Arabian can carry a heavy...",So kann auch ein kleinerer Araber einen schwer...,"[77, 98, 88]",87.66666666666667,"[-0.32967577825642896, 0.6715584827086242, 0.4...",0.276843,-0.450241
9966,6994,"Albany fled to France, then England, where he ...","Albany floh nach Frankreich, dann England, wo ...","[100, 79, 74]",84.33333333333333,"[0.2654514735602496, -0.13950363286393827, -0....",-0.219119,-0.230298
9968,6996,"In the late 1860s, the crinolines disappeared ...",In den späten 1860er Jahren verschwanden die K...,"[100, 93, 80]",91.0,"[0.2654514735602496, 0.22412516435479046, 0.32...",0.271088,-0.317499


In [19]:
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,6975,He vacated the NE0 title in April 2002 after r...,Nach seiner Rückkehr zum NE1 verließ er im Apr...,"[1, 13, 24]",12.666666666666666,"[-9.000885564931252, -10.29283318973243, -3.33...",-7.541931,-0.360573
1,2080,"NE2 and NE3, Proceedings of International Coll...","NE2 and NE3, Proceedings of International Coll...","[1, 1, 1]",1.0,"[-9.000885564931252, -8.00864789335575, -4.573...",-7.194496,-0.240925
2,6229,NE4 North Star Osage Ozark Trailblazers Pathfi...,NE4 North Star Osage Ozark Trailblazers Pathfi...,"[1, 1, 1]",1.0,"[-9.000885564931252, -7.556217303171075, -4.57...",-7.043685,-0.061312
3,2214,NE5.,NE5.,"[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.220932
4,1846,"NE6, NE7, IFWG PublishinNE8","NE6, NE7, NE8.","[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.129047
...,...,...,...,...,...,...,...,...
2967,6961,Sweden and Finland also augmented their roster...,Schweden und Finnland erweitert auch ihre Dien...,"[83, 98, 97]",92.66666666666667,"[0.7990540222688698, 0.20584462994058395, -0.0...",0.329851,-0.315033
2968,4410,East Pakistan objected and a protracted disput...,"Ostpakistan lehnte ab, und es kam zu einem lan...","[100, 94, 97]",97.0,"[0.2654514735602496, 0.3136118280255572, 0.504...",0.361346,-0.473948
2969,2247,The second group of historians present a class...,Die zweite Gruppe von Historikern präsentiert ...,"[100, 87, 77]",88.0,"[0.2654514735602496, 0.6211849487060245, -0.15...",0.244986,-0.289096
2970,3207,The central pairing of Maniche and NE3250 targ...,Die zentrale Paarung von Maniche und NE3250 zi...,"[100, 88, 94]",94.0,"[0.2654514735602496, -0.2233081539990433, 0.62...",0.222210,-0.256297
